In [ ]:
#imports
from ortools.linear_solver import pywraplp as OR
import pandas as pd
import math, itertools
import time
import random as rand

### Cereal Diet Problem (Lab 8)

$
\begin{align}
\min &\quad 3.8K + 4.2C \\
\text{subject to} &\quad 0.1K + 0.25C &\geq 1 \\
&\quad 1K + 0.25C &\geq 5 \\
&\quad 110K + 120C &\geq 400 \\
&\quad 0 \leq K, C
\end{align}
$

In [ ]:
# define the model
diet = OR.Solver('diet', OR.Solver.GLOP_LINEAR_PROGRAMMING);

In [ ]:
# decision variables
K = diet.NumVar(0, diet.infinity(), 'K');
C = diet.NumVar(0, diet.infinity(), 'C');

In [ ]:
# objective function
diet.Minimize(3.8*K + 4.2*C);

In [ ]:
# constraints
diet.Add(0.1*K + 0.25*C >= 1);
diet.Add(1*K + 0.25*C >= 5);
diet.Add(110*K + 120*C >= 400);

In [ ]:
# solve and print solution
diet.Solve()
print('Objective =', diet.Objective().Value())
print('Solution:')
for v in diet.variables():
    print(v.name(),':', v.solution_value())

### Stigler's Diet

*Based on CSV from http://excelcalculations.blogspot.com/2011/05/diet-problem-linear-programming.html*

In [ ]:
nutrients = pd.read_csv('data/nutrients.csv')
display(nutrients)
NUTRIENTS = []
bound = {}
for index, row in nutrients.iterrows():
    nutr = row['Nutrient']
    NUTRIENTS.append(nutr)
    bound.update({nutr : row['Bound']})

In [ ]:
foods = pd.read_csv('data/foods.csv')
display(foods)
FOODS = []
cost = {}
amt = {}
for index, row in foods.iterrows():
    food = row['Food']
    FOODS.append(food)
    cost.update({food : row['Cost']})
    for nutr in NUTRIENTS:
        amt.update({(food, nutr) : row[nutr]})

In [ ]:
# define model
m = OR.Solver('diet', OR.Solver.GLOP_LINEAR_PROGRAMMING)

In [ ]:
# decision variables
x = {}
for i in FOODS:
    x[i] = m.NumVar(0, m.infinity(), ('%s' % (i)))

In [ ]:
# objective function
m.Minimize(sum(x[i]*cost[i] for i in FOODS))

In [ ]:
# subject to: edge capacities 
for j in NUTRIENTS:
    m.Add(sum(x[i]*amt[i,j] for i in FOODS) >= bound[j])

In [ ]:
m.Solve()
print('Objective value =', m.Objective().Value())
print('Solution:')
for v in m.variables():
    print(v.name(),':', v.solution_value())

### Max Flow Implementation

In [ ]:
def MaxFlow(NODES,EDGES,capacity,source,sink):
    """Max flow model."""
    
    # define model
    m = OR.Solver('maxFlow', OR.Solver.GLOP_LINEAR_PROGRAMMING)
        
    # decision variables
    f = {}
    for i,j in EDGES:
        f[i,j] = m.NumVar(0, m.infinity(), ('(%s, %s)' % (i,j)))
            
    # objective function
    m.Maximize(sum(f[i,j] for i,j in EDGES if j == sink))
            
    # subject to: edge capacities 
    for i,j in EDGES:
        m.Add(f[i,j] <= capacity[i,j])
            
    # subject to: flow conservation 
    for k in NODES:
        if k != source and k != sink:
            flowIn = sum(f[i,j] for i,j in EDGES if j == k)
            flowOut = sum(f[i,j] for i,j in EDGES if i == k)
            m.Add(flowIn == flowOut)
            
    return m

In [ ]:
# EX: store max flow instance in two CSV files. Source and sink set manually.
nodes = pd.read_csv('data/nodes.csv') 
display(nodes)
edges = pd.read_csv('data/edges.csv') 
display(edges)

In [ ]:
# create node set
NODES = []
for index, row in nodes.iterrows():
    NODES.append(row['nodes'])
# set source and sink
source= 1
sink = 7

# create edge set
EDGES = []
capacity = {}
for index, row in edges.iterrows():
    edge = (row['tail'],row['head'])
    EDGES.append(edge)
    capacity.update({edge : row['capacity']})

In [ ]:
# create a model from abstract MaxFlow model 
mod= MaxFlow(NODES,EDGES,capacity,source,sink)
# solve model and print results
mod.Solve()
print('Objective value =', mod.Objective().Value())
print('Solution:')
for v in mod.variables():
    print(v.name(),':', v.solution_value())

### Branch and Bound (Incomplete)

$
\begin{align}
\max &\quad 5x_1 + 8x_2 \\
\text{subject to} &\quad x_1 + x_2 &\leq 6 \\
&\quad 5x_1 + 9x_2 &\leq 45 \\
&\quad x_1, x_2 &\leq 0 \\
&\quad x_1, x_2 &\text{integer} 
\end{align}
$

In [ ]:
# define the model
lp = OR.Solver('lp', OR.Solver.GLOP_LINEAR_PROGRAMMING);
# decision variables
x1 = lp.NumVar(0, diet.infinity(), 'x1');
x2 = lp.NumVar(0, diet.infinity(), 'x2');
# objective function
lp.Maximize(5*x1 + 8*x2);
# constraints
lp.Add(x1 + x2 <= 6);
lp.Add(5*x1 + 9*x2 <= 45);

In [ ]:
class Node:
    def __init__(self, lp, sol, z):
        self.lp = lp
        self.sol = sol
        self.z = z
    def lp(self):
        return lp
    def sol(self):
        return sol
    def z(self):
        return z 

In [ ]:
# returns a dictionary of variables to values
def Solution(lp):
    vals = {}
    for v in lp.variables():
        vals.update({v : v.solution_value()})
    return vals

In [ ]:
# branch on the given LP
def Branch(node):
    sol = node.sol
    for x in sol:
        val = sol[x]
        if not round(val,9).is_integer():
            branch = []
            leftLP = node.lp # not independent duplicate
            leftLP.Add(x <= math.floor(val)) 
            leftLP.Solve()
            if leftLP.FEASIBLE:
                zLeft = leftLP.Objective().Value()
                solLeft = Solution(leftLP)
                leftNode = Node(leftLP,solLeft,zLeft)
                branch.append(leftNode)                
            rightLP = node.lp
            rightLP.Add(x <= math.floor(val)) 
            rightLP.Solve()
            if rightLP.FEASIBLE:
                zRight = rightLP.Objective().Value()
                solRight = Solution(rightLP)
                rightNode = Node(rightLP,solRight,zRight)   
                branch.append(rightNode)
            return branch
    # solution is integral
    return node    

In [ ]:
# takes the LP relaxation and returns optimal integral solution
def BranchAndBound(lp):
    
    lp.Solve()
    z = lp.Objective().Value()
    sol = Solution(lp)
    root= Node(lp,sol,z)
    
    firstIncumbent = True
    incumbent = None
    active = [root]
    
    while len(active) > 0:
        # get an active subproblem
        subproblem = active.pop()
        
        if firstIncumbent == False and subproblem.z < incumbent.z:
            continue
        
        # branch
        branch = Branch(subproblem)
        if branch == subproblem:
            # must be integral
            # see if incumbent should be updated
            if firstIncumbent or subproblem.z > incumbent.z:
                firstIncumbent = False
                incumbent = subproblem
        else:
            for n in branch:  
                active.append(n)
    
    return incumbent

In [ ]:
opt = BranchAndBound(lp)
print('Objective: ', opt.z)
print('Solution:')
for x in opt.sol:
    print(x.name(), ':', opt.sol[x])

### Hotel Room Assignment (2019)

*Based on research by Henry Robbins and Sam Gutekunst*

In [ ]:
hotel = pd.read_csv('data/hotel.csv')
display(hotel.head())
ROOMS = []
rType = {}
for index, row in hotel.iterrows():
    room = row['number'].astype(int)
    ROOMS.append(room)
    rType.update({room : row['type']})

In [ ]:
arrivals = pd.read_csv('data/arrivals.csv')
display(arrivals.head())
GUESTS = []
gType = {}
for index, row in arrivals.iterrows():
    guest = row['id']
    GUESTS.append(guest)
    gType.update({guest : row['type']})

In [ ]:
weights = pd.read_csv('data/weights.csv')
display(weights.head())
weight = {}
for index, row in weights.iterrows():
    for r in ROOMS:
        weight.update({(row['guest'].astype(int),r): row[r]})

In [ ]:
PAIRS = list(itertools.product(GUESTS, ROOMS))

In [ ]:
# solver
m = OR.Solver('lp', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING);

In [ ]:
# decision variables
x = {}
for g,r in PAIRS:
    x[g,r] = m.IntVar(0,1,('(%s, %s)' % (g,r)))

In [ ]:
# objective function
m.Maximize(sum(x[g,r]*weight[g,r] for g,r in PAIRS)/len(GUESTS)) # set objective

In [ ]:
# subject to: every guest assigned one room
for g in GUESTS:
    m.Add(sum(x[g,r] for r in ROOMS) == 1)

In [ ]:
# subject to: every room assigned at most one guest
for r in ROOMS:
    m.Add(sum(x[g,r] for g in GUESTS) <= 1)

In [ ]:
# subject to: every guest assigned a room of appropriate type
for g in GUESTS:
    m.Add(sum(x[g,r]*rType[r] for r in ROOMS) >= gType[g])

In [ ]:
m.Solve()
print('Objective value =', m.Objective().Value())
print('Solution:')
for g,r in PAIRS:
    val = x[g,r].solution_value()
    if val == 1:
        print(x[g,r].name(),':',weight[g,r])
m.Objective().BestBound()